I will just write on each line a function to be called which we should try a net on. We could try combining functions later.

Functions 1:Standard MFCC

In [3]:
def mfcc_(song, n=20):
    return librosa.feature.mfcc(y = song, n_mfcc = n)


Function 2: MFCC without low

In [ ]:
def mfcc_cut(song, n=20, cutoff = 800:
    #remove bottom 800 hz of frequencies
    return librosa.feature.mfcc(y = song, f_min = cutoff, n_mfcc = n)

Function 3: LFCC with low

In [4]:
def LFCC(song, n=20):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

There might be a faster implementation of the above by replacing the for loop over time with multiplying by a matrix. Might be because I don't know how optimized scipy is at multiplying non square matrices.

Function 4:LFCC without low

In [5]:
def LFCC_no_low(song, n=20, cutoff = 800):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    cut = int(float(cutoff)/(librosa.fft_frequencies()[1]))
    D = D[cut:]
    #remove bottom 10.7*cutoff frequencies. The number 10.7 is the value of librosa.fft_frequencies()[1]
    DB = librosa.amplitude_to_db(D, ref=np.max)
    
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

Again maybe we should use a faster implementation as in function 2

Function 5:LFCC without low cleaned

In [7]:
def LFCC_no_low_clean(song, n=20, cutoff = 800):
    D = np.abs(librosa.stft(song, n_fft=n_fft,  hop_length=hop_length))
    cut = int(float(cutoff)/(librosa.fft_frequencies()[1]))
    D = D[cut:]
    DB = librosa.amplitude_to_db(D, ref=np.max)
    for x in range(0,DB.shape[0]):
        med = np.median(DB[x, :])
        DB[x, :] = DB[x, :]-med
        
    time = DB.shape[1]
    ans=np.zeros((n,time), dtype=float, order='C')
    for x in range(0,time):
        col = DB[:,x]
        ans[:,x] = (scipy.fft.dct(col)[0:n])
    return ans

Again implementation

Function 6:MFCC without low and cleaned

In [ ]:
def mfcc_cut_cleaned(song, n=20, cutoff = 800):
    Dmel = librosa.feature.melspectrogram(y=song, fmin = cutoff, n_fft=n_fft,  hop_length=hop_length)
    DBmel=librosa.power_to_db(Dmel)
    for x in range(0,DBmel.shape[0]):
        med = np.median(DBmel[x, :])
        DBmel[x, :] = DBmel[x, :]-med
    return librosa.feature.mfcc(S=DBmel, n_mfcc = n)